# Daily deaths for COVID19 at Madrid.

Simple kernel, to show how to use the dataset.

Contents:

 -  Load data.
 -  Fill empty data, and enrich with moving average
 -  R0 estimation
 -  Chart for UCI occupancy
 -  DAily UCI prediction
 -  Chart for daily death
 -  Compare deaths from today, last week and last month.
 -  Daily deaths prediction

 

In [ ]:
# Get updated_at day
! head -2 /kaggle/input/covid19-madrid/madrid_results.csv | tail -1 | cut -d, -f1

In [ ]:
from IPython.display import display, HTML
import os 
import pandas as pd
import glob 
import re 
from tqdm.notebook import tqdm
import warnings
import os
import numpy as np
import fbprophet

import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Show files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        

# Load data

In [ ]:
# Load data, sort it, and set its index 
df = pd.read_csv('/kaggle/input/covid19-madrid/madrid_results.csv')
df.sort_values('Fecha',inplace=True,ascending=False)
# Set date as index
df.set_index('Fecha', inplace=True, drop=True)



In [ ]:
### Calculate weekly deaths, last weeks

from datetime import timedelta
WEEKS_TO_CONSIDER=4
try :
    today =  pd.to_datetime(str(df['FALLECIDOS TOTALES'].index[0]))    
    array_deaths_last_weeks = [  df['FALLECIDOS TOTALES'].loc[str(today - timedelta(days=7*weeks)).replace(' 00:00:00','' )] for weeks in range(WEEKS_TO_CONSIDER+1) ] 
    reversed_array_deaths_last_weeks = array_deaths_last_weeks[::-1]
    array_deaths_by_week = np.diff(reversed_array_deaths_last_weeks)
    print(f"Madrid's Covid19 deaths by week, last {WEEKS_TO_CONSIDER} weeks: {array_deaths_by_week}")
    increments_array = [array_deaths_by_week[-1]/array_deaths_by_week[-2], 
         array_deaths_by_week[-2]/array_deaths_by_week[-3],
         array_deaths_by_week[-3]/array_deaths_by_week[-4]]
    increments_array=[x for x in reversed(increments_array) ]

    print(f"Madrid's Covid19 increments deaths by week : {increments_array}")

except : pass 


In [ ]:

# As we dont have complete data for all days , we fill emppty days using pandas interpolate function. 
# Doc at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
#
# First we define a wrapper function to do it

from datetime import datetime, date, time, timedelta

def interpolate_dataframe(df,freq):
    """Fill the empty days/hours in a dataframe with the quadratic interpolation"""
    if freq == 'H':
        rng = pd.date_range(df.index.min(), df.index.max() + pd.Timedelta(23, 'H'), freq='H')
    elif freq == 'D' :  
        rng = pd.date_range(
                        datetime.strptime(str(df.index.min())[:10]+' 00:00:00', "%Y-%m-%d %H:%M:%S") ,
                        datetime.strptime(str(df.index.max())[:10]+' 00:00:00', "%Y-%m-%d %H:%M:%S"), 
                        freq='D')
        df.index = pd.to_datetime(df.index)  
    df2 = df.reindex(rng)
    df = df2
    for column in df.columns :
        s = pd.Series(df[column])
        s.interpolate(method="linear", inplace =True)
        df[column] = pd.DataFrame([s]).T
    return df


In [ ]:
VENTANA_MEDIA_MOVIL=7

# Fill the empty days
# The caveat is that we can end with negative deaths in a specific day, but it happens also in the original data, due to regularizations
df = interpolate_dataframe(df,'D')
df.index.name = 'Fecha'
df = df.sort_values(by=['Fecha'], ascending=True)
#df.tail(8)



# Enrich columns with moving average columns, windows= last 7 days

In [ ]:
# Enrich columns with moving average columns, windows= last 7 days

df['HOSPITALES hoy'] = df['HOSPITALES'] - df['HOSPITALES'].shift(1)
df['CENTROS SOCIOSANITARIOS hoy'] = df['CENTROS SOCIOSANITARIOS'] - df['CENTROS SOCIOSANITARIOS'].shift(1)
df['FALLECIDOS TOTALES hoy'] = df['FALLECIDOS TOTALES'] - df['FALLECIDOS TOTALES'].shift(1)


df['MA CENTROS SOCIOSANITARIOS hoy'] = df['CENTROS SOCIOSANITARIOS hoy'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df['MA HOSPITALIZADOS'] = df['HOSPITALIZADOS'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df['MA FALLECIDOS TOTALES hoy'] = df['FALLECIDOS TOTALES hoy'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df['MA HOSPITALES hoy'] = df['HOSPITALES hoy'].rolling(window=VENTANA_MEDIA_MOVIL).mean()

df['MA PACIENTES UCI DIA'] = df['PACIENTES UCI DIA'].rolling(window=VENTANA_MEDIA_MOVIL).mean()

df = df.sort_index(ascending=False)

MADRID_POPULATION=6_642_000

df.sort_values('Fecha',inplace=True,ascending=True)
df['INCIDENCIA ACUMULADA'] = df['POSITIVOS'].rolling(window=14).sum()/6_642_000*100_000
df.sort_values('Fecha',inplace=True,ascending=False)


# Store a copy

df_master = df.copy()
df.head(10)

# SUMMARY

In [ ]:
from IPython.core.display import display, HTML


# MA Daily deaths (today, today_ma , yesterday_ma , last_week_ma )
# Hospitalized L
# ICUs (Last day, a week before)

def get_html_metric(today, yesterday, today_ma , yesterday_ma , last_week_ma, description):
    """ Return HTML for a metric"""
    color = "red" if today_ma > yesterday_ma else "green"

    html_code=f"""
        <div class="box">
          <div class="counter">{today}</div>
          <h2>{description}</h2>
            <table class="tg">
            <tbody>
              <tr>
                <th class="tg-0pky">yesterday</th>
                <th class="tg-0pky">{yesterday:.1f}</th>
              </tr>
              <tr>
                <th class="tg-0pky">MA today</th>
                <th class="tg-0pky">{today_ma:.1f}</th>
              </tr>
              <tr>
                <td class="tg-0lax">MA yesterday</td>
                <td class="tg-0lax">{yesterday_ma:.1f}</td>
              </tr>
              <tr>
                <td class="tg-0lax">MA last week</td>
                <td class="tg-0lax"> {last_week_ma:.1f}</td>
              </tr>
            </tbody>
            </table>
        </div>
        <style>
            body{{
              background: rgb(200, 0, 0);
              text-align: left;
              font-family: 'Helvetica';
            }}

            .box{{
              margin: 45px auto 0 auto;
              width: 180px;
              height: 180px;
              line-height: 180px;
              border-radius: 50%;
              background: {color};
              text-align: center;
              font-family: 'Helvetica';
            }}

            .counter{{
              font-size: 60px;

            }}
</style>
        """
    #html_code+=f"""<h2>{description}</h2>"""
    return html_code


today =     int(df.iloc[0:1]['FALLECIDOS TOTALES hoy'].values[0])
yesterday = int(df.iloc[1:2]['FALLECIDOS TOTALES hoy'].values[0])
today_ma = df.iloc[0:1]['MA FALLECIDOS TOTALES hoy'].values[0]
yesterday_ma = df.iloc[1:2]['MA FALLECIDOS TOTALES hoy'].values[0]
last_week_ma = df.iloc[7:8]['MA FALLECIDOS TOTALES hoy'].values[0]
description="Daily Deaths"
html_death = get_html_metric(today,yesterday ,today_ma , yesterday_ma , last_week_ma, description)

today = int(df.iloc[0:1]['HOSPITALIZADOS'].values[0])
yesterday = int(df.iloc[1:2]['HOSPITALIZADOS'].values[0])

today_ma = df.iloc[0:1]['MA HOSPITALIZADOS'].values[0]
yesterday_ma = df.iloc[1:2]['MA HOSPITALIZADOS'].values[0]
last_week_ma = df.iloc[7:8]['MA HOSPITALIZADOS'].values[0]
description="Hospitalized"
html_hospitalized = get_html_metric(today,yesterday ,today_ma , yesterday_ma , last_week_ma, description)

today = int(df.iloc[0:1]['PACIENTES UCI DIA'].values[0])
yesterday = int(df.iloc[1:2]['PACIENTES UCI DIA'].values[0])
today_ma = df.iloc[0:1]['MA PACIENTES UCI DIA'].values[0]
yesterday_ma = df.iloc[1:2]['MA PACIENTES UCI DIA'].values[0]
last_week_ma = df.iloc[7:8]['MA PACIENTES UCI DIA'].values[0]
description="UCI"
html_uci = get_html_metric(today,yesterday ,today_ma , yesterday_ma , last_week_ma, description)


today = int(df.iloc[0:1]['INCIDENCIA ACUMULADA'].values[0])
yesterday = int(df.iloc[1:2]['INCIDENCIA ACUMULADA'].values[0])
today_ma = today
yesterday_ma = yesterday
last_week_ma = df.iloc[7:8]['INCIDENCIA ACUMULADA'].values[0]
description="Cumulative Incidence"
html_ci = get_html_metric(today,yesterday ,today_ma , yesterday_ma , last_week_ma, description)


metrics = f"""
<table class="tg">
<thead>
  <tr>
    <th>{html_ci}</th>
    <th>{html_hospitalized}</th>
    <th>{html_uci}</th>
    <th>{html_death}</th>
  </tr>
</thead>
</table>
""" 
c2 = display(HTML(metrics))


# R0 estimation at Madrid

In [ ]:
# Estimate a R0, using the number of deaths instead of infected0.0

from datetime import datetime, timedelta
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

df = df_master

def calcular_estimaciones_R0(df):
    def calcular_R0_dia(dia,df):
        dia_semana_anterior = dia - timedelta(days=7)
        return dia,df.loc[dia:dia - timedelta(days=6)]['FALLECIDOS TOTALES hoy'].sum() / df.loc[dia- timedelta(days=7):dia - timedelta(days=13)]['FALLECIDOS TOTALES hoy'].sum() 

    VENTANA_MEDIA_MOVIL=7
    
    df_R0_estimated = pd.DataFrame([calcular_R0_dia(dia,df) for dia in df.index[0:150]],columns=['Fecha','R0_estimated'])
    
    df_R0_estimated = df_R0_estimated.sort_values(by=['Fecha'], ascending=True)
    df_R0_estimated['R0_estimated Moving Average'] = df_R0_estimated['R0_estimated'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
    df_R0_estimated = df_R0_estimated.sort_values(by=['Fecha'], ascending=False)
    df_R0_estimated.set_index('Fecha', inplace=True, drop=True)
    return df_R0_estimated

df= calcular_estimaciones_R0(df_master)
#df=df[['R0_estimated']]

    
chart_df=df[df.columns[-3:]]
chart_df.plot(legend=True,figsize=(13.5,9), marker='o')

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
plt.xticks(rotation=45)

ax = plt.gca()
ax.axhline(1, color='r',linestyle =  ':' ) 


ax.set_title("R0 Estimated, Madrid")
ax.set_ylim(ymin=0)

plt.show()

df.head(21).style.format ({ c : "{:20,.3f}" for c in df.columns }).background_gradient(cmap='Wistia', )


In [ ]:
import pandas as pd
import io
import matplotlib.dates as mdates
from matplotlib import pyplot as plt

df = df_master


df = df.sort_values(by=['Fecha'], ascending=True)

df['MA PACIENTES UCI DIA'] = df['PACIENTES UCI DIA'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df['ratio'] =  df['MA PACIENTES UCI DIA']/ df['MA FALLECIDOS TOTALES hoy']
df = df.sort_values(by=['Fecha'], ascending=False)

chart_df=df[['MA FALLECIDOS TOTALES hoy', 'MA PACIENTES UCI DIA', 'ratio']]


chart_df.plot(legend=True,figsize=(13.5,9), marker='o')

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=15))
plt.xticks(rotation=45)

ax = plt.gca()
ax.yaxis.tick_right()
ax.set_title("Madrid COVID19 ICU occupancy and deaths. MA "+str(VENTANA_MEDIA_MOVIL)+" days.")
ax.set_ylim(ymin=0)
plt.grid(color='silver', linestyle='-', linewidth=2, axis='y')

plt.show()
df['PACIENTES UCI DIFERENCIA DIA']=df['PACIENTES UCI DIA'] -df['PACIENTES UCI DIA'].shift(-1)
df['PACIENTES UCI DIFERENCIA DIA PORCENTAGE']=(
    (df['PACIENTES UCI DIA'] / df['PACIENTES UCI DIA'].shift(-1)) -1 ) *100
df = df.sort_values(by=['Fecha'], ascending=True)
df['MA PACIENTES UCI DIFERENCIA DIA PORCENTAGE'] = df['PACIENTES UCI DIFERENCIA DIA PORCENTAGE'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df = df.sort_values(by=['Fecha'], ascending=False)

df[['PACIENTES UCI DIA','PACIENTES UCI DIFERENCIA DIA','PACIENTES UCI DIFERENCIA DIA PORCENTAGE','MA PACIENTES UCI DIFERENCIA DIA PORCENTAGE','MA PACIENTES UCI DIA','MA FALLECIDOS TOTALES hoy']].head(21).style.format ({ c : "{:20,.3f}"  for c in df.columns })

# Waves comparision


In [ ]:
from datetime import datetime

def get_ola(df_master,limite_inferior,limite_superior,columna):
    df = df_master
    VENTANA_MEDIA_MOVIL_COMP_OLAS=15

    date1 = datetime.strptime(limite_inferior, "%Y-%m-%d").strftime("%Y-%m-%d")
    date2 = datetime.strptime(limite_superior, "%Y-%m-%d").strftime("%Y-%m-%d")

    df = df.loc[date2:date1][[f"{columna}"]]
    df = df.sort_values(by=['Fecha'], ascending=True)
    return df[columna].values


def calculate_df_comparacion(arr2,arr3,arr4,arr5) : 
    max_dias = max(len(arr2),len(arr3),len(arr4),len(arr5))

    results = []
    for dia in range(max_dias):
        my_dict = {}
        try: my_dict[ "Second wave"] =  arr2[dia] 
        except: pass
        try: my_dict[ "Third wave" ] =  arr3[dia] 
        except: pass
        try: my_dict[ "Fourth wave "] =  arr4[dia] 
        except: pass
        try: my_dict[ "Fifth wave" ] =  arr5[dia] 
        except: pass
        dict_has_values = my_dict
        if dict_has_values : 
            results.append(my_dict)    
    df = pd.DataFrame(results)        
    return df



columna = 'PACIENTES UCI DIA'
arr5 = get_ola(df_master,"2021-07-16",datetime.today().strftime('%Y-%m-%d'),columna)
arr4 = get_ola(df_master,"2021-03-26","2021-05-24",columna)
arr3 = get_ola(df_master,"2021-01-01","2021-03-17",columna)
arr2 = get_ola(df_master,"2020-08-07","2020-12-07",columna)

df_icu = calculate_df_comparacion(arr2,arr3,arr4,arr5)

columna = 'MA FALLECIDOS TOTALES hoy'
arr5 = get_ola(df_master,"2021-07-23",datetime.today().strftime('%Y-%m-%d'),columna)
arr4 = get_ola(df_master,"2021-04-02","2021-06-01",columna)
arr3 = get_ola(df_master,"2021-01-08","2021-03-25",columna)
arr2 = get_ola(df_master,"2020-08-15","2020-12-14",columna)
df_deaths = calculate_df_comparacion(arr2,arr3,arr4,arr5)

columna = 'HOSPITALIZADOS'
arr5 = get_ola(df_master,"2021-07-09",datetime.today().strftime('%Y-%m-%d'),columna)
arr4 = get_ola(df_master,"2021-03-19","2021-05-27",columna)
arr3 = get_ola(df_master,"2020-12-25","2021-03-10",columna)
arr2 = get_ola(df_master,"2020-08-01","2020-12-01",columna)
df_hospitalized = calculate_df_comparacion(arr2,arr3,arr4,arr5)

columna = 'INCIDENCIA ACUMULADA'
arr5 = get_ola(df_master,"2021-07-02",datetime.today().strftime('%Y-%m-%d'),columna)
arr4 = get_ola(df_master,"2021-03-12","2021-05-20",columna)
arr3 = get_ola(df_master,"2020-12-18","2021-03-03",columna)
arr2 = get_ola(df_master,"2020-07-25","2020-11-23",columna)
df_ia = calculate_df_comparacion(arr2,arr3,arr4,arr5)

In [ ]:
from matplotlib.pyplot import figure


fig, ax = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(11.5, 7.5)
fig.suptitle('Comparison by wave', fontsize=16)
ax[0][0].plot(df_ia)
ax[0][0].set_title("Accumulative incidence")

ax[0][1].plot(df_hospitalized)
ax[0][1].set_title("Hospitalizations")
ax[0][1].legend(df_deaths)

ax[1][0].plot(df_icu)
ax[1][0].set_title("ICUs occupied")

ax[1][1].plot(df_deaths)
ax[1][1].set_title("Daily deaths")

plt.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

TITLES = ["Cumulative incidence", "Hospitalizations","ICU Beds","Daily deaths"]
DFS= ((df_ia, df_hospitalized),(df_icu,df_deaths))
colors = [ "blue", "orange", "green", "red"]

fig = make_subplots(rows=2, cols=2, subplot_titles=TITLES)
#subplot_titles=TITLES
fig.update_layout(
  plot_bgcolor = 'rgba(0,0,0,0)',
  hovermode = "x",
  showlegend = False,
  font = dict(
    family = "Helvetica",
    size = 16,
    color = "#0072B2"
  )
)

fig.update_yaxes(zeroline = False, showline = True, showgrid = False, spikemode = 'toaxis', spikesnap = 'cursor', spikethickness = 2, spikedash = 'solid', spikecolor = 'Grey', color = 'LightGrey', linecolor = 'Grey', hoverformat = ".0f")
fig.update_xaxes(showline = True, 
                 showgrid = False, 
                 spikemode = 'toaxis', 
                 spikesnap = 'cursor', 
                 spikethickness = 2, 
                 spikedash = 'solid', 
                 spikecolor = 'Grey', 
                 color = 'LightGrey', 
                 linecolor = 'Grey', 
                 hoverformat = ".0f", 
                 rangeslider = {'visible': False}, 
                 rangeselector = {'visible': False},
                 title_font_size=12)

df_plot = df_hospitalized

for row in range(1,3) :  
    for column in range(1,3) :  
        df_plot = DFS[row-1][column-1]
        for j,column_wave in enumerate(df_plot.columns):
            fig.append_trace(go.Scatter(
                mode = "lines+text",
                line=dict(color=colors[j]),
                x=df_plot.index.values,
                y=df_plot[column_wave].values,
                name=column_wave,
                showlegend = False,           
            ), row=row, col=column)


fig.update_layout(height=600, width=800, title_text="Madrid, Comparison between COVID19 waves")
fig.show()

# ICU Prediction

In [ ]:
# Python
import fbprophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go

m = fbprophet.Prophet(changepoint_prior_scale = 0.15)

df = df_master


df = df.sort_values(by=['Fecha'], ascending=True)

df['MA PACIENTES UCI DIA'] = df['PACIENTES UCI DIA'].rolling(window=VENTANA_MEDIA_MOVIL).mean()
df=df[['MA PACIENTES UCI DIA','MA FALLECIDOS TOTALES hoy']]
df = df.sort_values(by=['Fecha'], ascending=False)


df = df[['MA PACIENTES UCI DIA']]
df = df.dropna()

df['ds'] = df.index
df = df.rename(columns = {
  'MA PACIENTES UCI DIA': 'y'
})
df = df[['ds', 'y']]
df = df.sort_values(by = 'ds').tail(21)

m.fit(df)
future = m.make_future_dataframe(periods = 28)
forecast = m.predict(future)

fig = plot_plotly(m, forecast)

fig.update_layout(
  plot_bgcolor = 'rgba(0,0,0,0)',
  title = "Madrid, Covid19 daily ICU beds, 3 weeks prediction",
  yaxis_title = "ICU beds occupied, moving average",
  hovermode = "x",
  showlegend = False,
  xaxis_tickformat = "%b %d",
  font = dict(
    family = "Helvetica",
    size = 18,
    color = "#0072B2"
  )
)


fig.update_yaxes(zeroline = False, showline = True, showgrid = False, spikemode = 'toaxis', spikesnap = 'cursor', spikethickness = 2, spikedash = 'solid', spikecolor = 'Grey', color = 'LightGrey', linecolor = 'Grey', hoverformat = ".0f")
fig.update_xaxes(showline = True, showgrid = False, spikemode = 'toaxis', spikesnap = 'cursor', spikethickness = 2, spikedash = 'solid', spikecolor = 'Grey', color = 'LightGrey', linecolor = 'Grey', hoverformat = "%b %d", title = "", rangeslider = {
  'visible': False
}, rangeselector = {
  'visible': False
})
plt.show()
fig

# Chart for daily deaths at Madrid

In [ ]:

from matplotlib import colors

def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

df = df_master
df = df.head(30)
df.style.format ({ c : "{:20,.0f}" for c in df.columns }).background_gradient(cmap='Wistia', subset= df.columns[-4:] )



## Daily deaths prediction

Using prophet.

In [ ]:
df = df_master

df = df[['FALLECIDOS TOTALES hoy']]
df = df.dropna()

df['ds'] = df.index
df = df.rename(columns = {'FALLECIDOS TOTALES hoy':'y'})
df = df[['ds','y']]
df = df.sort_values(by='ds').tail(21)
df

In [ ]:
from IPython.display import display, HTML
HTML("<h2>Compare today, against previous week and month </h2>")    

In [ ]:
df = df_master
try : 
    pd.concat([df.head(1).tail(1) , df.head(8).tail(1) , df.head(30).tail(1)]).astype(int)[['MA HOSPITALES hoy','MA CENTROS SOCIOSANITARIOS hoy','MA FALLECIDOS TOTALES hoy']].style.format ({ c : "{:20,.0f}" for c in df.columns }).background_gradient(cmap='Wistia', subset= df.columns[-3:] )
    
except: pass 

In [ ]:
HTML("<h2>Average daily death, from the time the 'Comunidad of Madrid' began to publish data  </h2>")

In [ ]:
df = df_master
df = pd.DataFrame((df.head(1).max(axis=0) - df.tail(1).max(axis=0) ) / df.shape[0] ).T[['HOSPITALES','DOMICILIOS','CENTROS SOCIOSANITARIOS','OTROS LUGARES','FALLECIDOS TOTALES']]
df.style.format ({ c : "{:20,.0f}" for c in df.columns }).background_gradient(cmap='Wistia'  )

# Prediction for COVID daily deaths at Madrid

For a quick prediction, we use [fbprophet](https://facebook.github.io/prophet/), the forecasting time series library made by facebook.


In [ ]:
# Python
import fbprophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go

m = fbprophet.Prophet(changepoint_prior_scale = 0.15)

df = df_master

df = df[['MA FALLECIDOS TOTALES hoy']]
df = df.dropna()

df['ds'] = df.index
df = df.rename(columns = {
  'MA FALLECIDOS TOTALES hoy': 'y'
})
df = df[['ds', 'y']]
df = df.sort_values(by = 'ds').tail(245)

m.fit(df)
future = m.make_future_dataframe(periods = 28)
forecast = m.predict(future)

fig = plot_plotly(m, forecast)


fig.update_layout(
  plot_bgcolor = 'rgba(0,0,0,0)',
  title = "Madrid, Covid19 daily deaths, 3 weeks prediction",
  yaxis_title = "Daily deaths, moving average",
  hovermode = "x",
  showlegend = False,
  xaxis_tickformat = "%b %d",
  font = dict(
    family = "Helvetica",
    size = 18,
    color = "#0072B2"
  )
)

fromValue = 0
toValue = df.y.max() * 1.30

fig.update_layout(yaxis=dict(range=[fromValue,toValue]))


fig.update_yaxes(zeroline = False, showline = True, showgrid = False, spikemode = 'toaxis', spikesnap = 'cursor', spikethickness = 2, spikedash = 'solid', spikecolor = 'Grey', color = 'LightGrey', linecolor = 'Grey', hoverformat = ".0f")
fig.update_xaxes(showline = True, showgrid = False, spikemode = 'toaxis', spikesnap = 'cursor', spikethickness = 2, spikedash = 'solid', spikecolor = 'Grey', color = 'LightGrey', linecolor = 'Grey', hoverformat = "%b %d", title = "", rangeslider = {
  'visible': False
}, rangeselector = {
  'visible': False
})
# plt.show()


def  annotate_chart(event_date,line_color,event_text):
    long_event_date=event_date +' 00:00:00'    
    if df.index[0]<pd.to_datetime(long_event_date):
        fig.add_trace(go.Scatter(
          x = [pd.to_datetime(long_event_date), pd.to_datetime(long_event_date)],
          y = [forecast.query(" ds == @event_date ").yhat.values[0], forecast.query(" ds == @event_date ").yhat.values[0]+( forecast.yhat_upper.max()/5)],
          mode = "lines+text",
          line_color =line_color,
          name = "",
          text = ["", event_text],
          textposition = "top center",
          textfont = dict(
            family = "Helvetica",
            size = 18,
            color = line_color
          )
        ))


annotate_chart(event_date = "2020-09-04",
               line_color = '#b20038',
               event_text = "Reopening schools")
lockdown_color = '#7a00b2'

annotate_chart(event_date = "2020-09-18",
               line_color = lockdown_color,
               event_text = "Partial lockdown")

annotate_chart(event_date = "2020-09-25",
               line_color = lockdown_color,
               event_text = "Partial lockdow improved")

annotate_chart(event_date = "2020-10-02",
               line_color = lockdown_color,
               event_text = "Almost full lockdown")

annotate_chart(event_date = "2020-10-30",
               line_color = lockdown_color,
               event_text = "Perimetral lockdown")


fig

## Thanks for watching!